So previously I discussed about aggressive agents on https://www.kaggle.com/c/hungry-geese/discussion/216976

This is my current take on it, I think it might be useful for those who are testing their ML agents out there.

# Agent: Risk Averse Greedy

In [ ]:
%%writefile risk_averse_greedy.py

from kaggle_environments.envs.hungry_geese.hungry_geese import Observation, Configuration, Action, row_col
import numpy as np

def get_nearest_cells(x,y):
    # returns all cells reachable from the current one
    result = []
    for i in (-1,+1):
        result.append(((x+i+7)%7, y))
        result.append((x, (y+i+11)%11))
    return result

def find_closest_food(table):
    # returns the first step toward the closest food item
    new_table = table.copy()
    
    updated = False
    for roll, axis, code in [
        (1, 0, 1),
        (-1, 0, 2),
        (1, 1, 3),
        (-1, 1, 4)
    ]:

        shifted_table = np.roll(table, roll, axis)
        
        if (table == -2).any() and (shifted_table[table == -2] == -3).any(): # we have found some food at the first step
            return code
        else:
            mask = np.logical_and(new_table == 0,shifted_table == -3)
            if mask.sum() > 0:
                updated = True
            new_table += code * mask
        if (table == -2).any() and shifted_table[table == -2][0] > 0: # we have found some food
            return shifted_table[table == -2][0]
        
        # else - update new reachible cells
        mask = np.logical_and(new_table == 0,shifted_table > 0)
        if mask.sum() > 0:
            updated = True
        new_table += shifted_table * mask

    # if we updated anything - continue reccurison
    if updated:
        return find_closest_food(new_table)
    # if not - return some step
    else:
        return table.max()

def agent(obs_dict, config_dict):
    """This agent always moves toward observation.food[0] but does not take advantage of board wrapping"""
    observation = Observation(obs_dict)
    configuration = Configuration(config_dict)
    player_index = observation.index
    player_goose = observation.geese[player_index]
    player_head = player_goose[0]
    player_row, player_column = row_col(player_head, configuration.columns)


    table = np.zeros((7,11))
    # 0 - emply cells
    # -1 - obstacles
    # -2 - food
    # -3 - head
    # 1,2,3,4 - reachable on the current step cell, number is the id of the first step direction
    
    legend = {
        1: 'SOUTH',
        2: 'NORTH',
        3: 'EAST',
        4: 'WEST'
    }
    
    # let's add food to the map
    for food in observation.food:
        x,y = row_col(food, configuration.columns)
        table[x,y] = -2 # food
        
    # let's add all cells that are forbidden
    for i in range(4):
        opp_goose = observation.geese[i]
        if len(opp_goose) == 0:
            continue
        for n in opp_goose[:-1]:
            x,y = row_col(n, configuration.columns)
            table[x,y] = -1 # forbidden cells
        if i != player_index:
            x,y = row_col(opp_goose[0], configuration.columns)
            possible_moves = get_nearest_cells(x,y) # head can move anywhere
            for x,y in possible_moves:
                table[x,y] = -1 # forbidden cells

        
    # let's add head position
    x,y = row_col(player_head, configuration.columns)
    table[x,y] = -3
    
    # the first step toward the nearest food
    step = int(find_closest_food(table))
    
    # if there is not available steps make random step
    if step not in [1,2,3,4]:
        step = np.random.randint(4) + 1
    
    return legend[step]

# Agent: Greedy + food hoarding

In [ ]:
%%writefile greedyhoarder.py

from kaggle_environments.envs.hungry_geese.hungry_geese import Observation, Configuration, Action, row_col
from random import choice, sample, seed, randint

def random_agent():
    return choice([action for action in Action]).name


def translate(position: int, direction: Action, columns: int, rows: int):
    row, column = row_col(position, columns)
    row_offset, column_offset = direction.to_row_col()
    row = (row + row_offset) % rows
    column = (column + column_offset) % columns
    return row * columns + column


def adjacent_positions(position: int, columns: int, rows: int):
    return [
        translate(position, action, columns, rows)
        for action in Action
    ]


def min_distance(position: int, food: [int], columns: int):
    row, column = row_col(position, columns)
    return min(
        abs(row - food_row) + abs(column - food_column)
        for food_position in food
        for food_row, food_column in [row_col(food_position, columns)]
    )

last_pos = -1

def agent(observation, configuration):
    global last_pos
    observation = Observation(observation)
    configuration = Configuration(configuration)
    rows, columns = configuration.rows, configuration.columns
    
    food = observation.food
    geese = observation.geese
    
    player_index = observation.index
    my_goose = geese[player_index]
    my_tail = [my_goose[-1]]
    position = geese[observation.index][0]
    
    opponents = [
        goose
        for index, goose in enumerate(geese)
        if index != observation.index and len(goose) > 0
    ]

    # Don't move adjacent to any heads
    head_adjacent_positions = {
        opponent_head_adjacent
        for opponent in opponents
        for opponent_head in [opponent[0]]
        for opponent_head_adjacent in adjacent_positions(opponent_head, columns, rows)
    }
    # Don't move into any bodies
    bodies = {position for goose in geese for position in goose[0:-1]}
    # Don't move into tails of heads that are adjacent to food
    tails = {
        opponent[-1]
        for opponent in opponents
        for opponent_head in [opponent[0]]
        if any(
            adjacent_position in food
            # Head of opponent is adjacent to food so tail is not safe
            for adjacent_position in adjacent_positions(opponent_head, columns, rows)
        )
    }

    # Move to the closest food
    position = geese[observation.index][0]
    if len(geese[player_index]) > 8: #HOARDING POWER
        factions = {
        action: min_distance(new_position, food, columns)
        for action in Action
        for new_position in [translate(position, action, columns, rows)]
        if (
            new_position not in last_pos and
            new_position not in bodies and
            new_position not in tails and
            new_position not in food
        )
        if (
            new_position not in (food and my_tail)
        )
        }
        if any(factions):
            new_factions = {
            action: min_distance(new_position, food, columns)
            for action in Action
            for new_position in [translate(position, action, columns, rows)]
            if (
                new_position not in last_pos and
                new_position not in bodies and
                new_position not in tails and
                new_position not in food and
                new_position not in head_adjacent_positions
            )
            if (
                new_position not in (food and my_tail)
            )
            }
            if any(new_factions):
                last_pos = {geese[player_index][0]}
                step = min(new_factions, key=new_factions.get)
                #print(step)
                steps = [action for action in new_factions.keys() if(new_factions[action] == min(new_factions.values()))]
                #print(steps)
                seed(None,randint(1,2))
                step = choice(steps)
                #print(new_factions)

                #print(step)
                #print(step.name,"food hoarding")
                seed(1)
                return step.name
    actions = {
        action: min_distance(new_position, food, columns)
        for action in Action
        for new_position in [translate(position, action, columns, rows)]
        if (
            new_position not in head_adjacent_positions and
            new_position not in bodies and
            new_position not in tails
        )
    }

    if any(actions):
        last_pos = {geese[player_index][0]}
        return min(actions, key=actions.get).name

    return random_agent()

# Agent: Hard coded Food hoarder

I havent been around for long after Kaggle updated their environment. Something does seem a bit different now, will look into it when I have the time. For the time though, enjoy the C-shaped traps that is made when avoiding food.

In [ ]:
%%writefile besthoarder.py

from kaggle_environments.envs.hungry_geese.hungry_geese import Observation, Configuration, Action, row_col
from random import choice, sample, randint, seed

import time

def random_agent():
    seed(None,randint(1,2))
    step = choice([action for action in Action])
    seed(1)
    print(step.name,"bugged")
    return step.name


def translate(position: int, direction: Action, columns: int, rows: int):
    row = position // columns
    column = position % columns
    row_offset, column_offset = direction.to_row_col()
    row = (row + row_offset) % rows
    column = (column + column_offset) % columns
    return row * columns + column


def adjacent_positions(position: int, columns: int, rows: int):
    return [
        translate(position, action, columns, rows)
        for action in Action
    ]


def min_distance(position: int, food: [int], columns: int):
    row, column = row_col(position, columns)
    return min(
        abs(row - food_row) + abs(column - food_column)
        for food_position in food
        for food_row, food_column in [row_col(food_position, columns)]
    )

last_pos = {}

def agent(observation, configuration):
    global last_pos
    try:
        print("===ROUND ",observation.step+1,"===")
        time_start = time.perf_counter()
        observation = Observation(observation)
        configuration = Configuration(configuration)
        rows, columns = configuration.rows, configuration.columns

        food = observation.food
        geese = observation.geese
        player_index = observation.index
        my_goose = geese[player_index]
        my_tail = [my_goose[-1]]
        position = geese[observation.index][0]
        
        print("At row: ",position//11," column: ", position%11," food at: ", food)
        
        # Don't move into any bodies
        bodies = {position for goose in geese for position in goose[0:-1]}
        # Opponent definition
        opponents = [
            goose
            for index, goose in enumerate(geese)
            if index != observation.index and len(goose) > 0
        ]
        # Don't move into tails of heads that are adjacent to food
        tails = {
            opponent[-1]
            for opponent in opponents
            for opponent_head in [opponent[0]]
            if any(
                adjacent_position in food
                # Head of opponent is adjacent to food so tail is not safe
                for adjacent_position in adjacent_positions(opponent_head, columns, rows)
            )
        }        
        # Don't move adjacent to any heads
        head_adjacent_positions = {
            opponent_head_adjacent
            for opponent in opponents
            for opponent_head in [opponent[0]]
            for opponent_head_adjacent in adjacent_positions(opponent_head, columns, rows)
        }
        
        myself = [my_goose]

        keep_dist = {
            keep_dist
            for head_adjacent_position in head_adjacent_positions
            for keep_dist in adjacent_positions(head_adjacent_position, columns, rows)
        }
        
        """Hard coding movement here"""
        
        #MOVEMENT TIME
        if len(geese[player_index]) > 3 and len(geese[player_index]) < 9: #TURTLE POWER
            tactions = [
            action
            for action in Action
            for new_position in [translate(position, action, columns, rows)]
            if (
                new_position in my_tail and
                new_position not in last_pos and
                new_position not in food #Checking for picking up food, which would kill my goose 
            )
            ]
            if any(tactions): # CHeck if TURTLE POWER is valid
                new_tactions = [
                action
                for action in Action
                for new_position in [translate(position, action, columns, rows)]
                if (
                    new_position in my_tail and
                    new_position not in last_pos and
                    new_position not in food and
                    new_position not in head_adjacent_positions #Looking for goose walking into my tail
                )
                ]
                if any(new_tactions):
                    last_pos = {geese[player_index][0]}
                    print(new_tactions[0].name, "turtling")
                    seed(1)
                    return new_tactions[0].name
            
        # Checking if viable steps exist in next 4 turns
        possible_actions = {
            action: new_position5
            for action in Action
            for new_position1 in [translate(position, action, columns, rows)]
            if (
                new_position1 not in last_pos and
                new_position1 not in bodies and
                new_position1 not in tails
            )
            for action2 in Action
            for new_position2 in [translate(new_position1, action2, columns, rows)]
            if (
                new_position2 not in [geese[player_index][0]] and
                new_position2 not in bodies and
                new_position2 not in tails
            )
            for action3 in Action
            for new_position3 in [translate(new_position2, action3, columns, rows)]
            if (
                new_position3 not in [new_position1] and
                new_position3 not in bodies and
                new_position3 not in tails
            )
            for action4 in Action
            for new_position4 in [translate(new_position3, action4, columns, rows)]
            if (
                new_position4 not in [new_position2] and
                new_position4 not in bodies and
                new_position4 not in tails
            )
            for action5 in Action
            for new_position5 in [translate(new_position4, action5, columns, rows)]
            if (
                new_position5 not in [new_position3] and
                new_position5 not in bodies and
                new_position5 not in tails
            )
        }
        
        #Food turtling/Food hoarding/Passive Aggro Geese whatever you call it
        
        if len(geese[player_index]) > 7:
            factions = {
            action: min_distance(new_position, food, columns)
            for action in possible_actions
            for new_position in [translate(position, action, columns, rows)]
            if (
                new_position not in last_pos and
                new_position not in bodies and
                new_position not in tails and
                new_position not in food # Will avoid food and circle around it
            )
            if (
                new_position not in (food and my_tail)
            )
            }
            if any(factions):
                new_factions = {
                action: min_distance(new_position, food, columns)
                for action in possible_actions
                for new_position in [translate(position, action, columns, rows)]
                if (
                    new_position not in last_pos and
                    new_position not in bodies and
                    new_position not in tails and
                    new_position not in food and
                    new_position not in head_adjacent_positions
                )
                if (
                    new_position not in (food and my_tail)
                )
                }
                if any(new_factions):
                    last_pos = {geese[player_index][0]}
                    step = min(new_factions, key=new_factions.get)
                    print(step)
                    steps = [action for action in new_factions.keys() if(new_factions[action] == min(new_factions.values()))]
                    print(steps)
                    seed(None,randint(1,2))
                    step = choice(steps)
                    print(new_factions)
                    
                    print(step)
                    print(step.name,"food hoarding")
                    seed(1)
                    return step.name
            
        #print(possible_actions)
        actions = {
            action: min_distance(new_position, food, columns)
            for action in possible_actions
            for new_position in [translate(position, action, columns, rows)]
            if (
                new_position not in last_pos and
                new_position not in bodies and
                new_position not in tails
            )
            if (
                new_position not in (food and my_tail)
            )
        }
        if any(actions):
            new_actions = {
                action: min_distance(new_position, food, columns)
                for action in possible_actions
                for new_position in [translate(position, action, columns, rows)]
                if (
                    new_position not in last_pos and
                    new_position not in bodies and
                    new_position not in tails and
                    new_position not in head_adjacent_positions
                )
                if (
                    new_position not in (food and my_tail)
                )
            }
            if any(new_actions):
                actions = new_actions 

        #print(actions, '*')
        last_pos = {geese[player_index][0]}

        time_taken = time.perf_counter() - time_start
        #print(actions, time_taken)

        if any(actions):
            step = min(actions, key=actions.get)
            steps = [action for action in actions.keys() if(actions[action] == min(actions.values()))]
            seed(None,randint(1,2))
            step = choice(steps)
            print(step.name,"norm")
            seed(1)
            return step.name
        
    except Exception as e:
        print("ERROR:",e)
        return random_agent()
    
    return random_agent()

In [ ]:
import collections

from kaggle_environments import evaluate, make, utils

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sn

In [ ]:
env = make("hungry_geese", debug=False) # You dont want to see the logs

In [ ]:
env.reset()
env.run(
    [
        "besthoarder.py", #white
        "greedyhoarder.py",
        "risk_averse_greedy.py",
        "greedyhoarder.py",
    ],
)
(env.render(mode="ipython", width=800, height=700))